In [1]:
input_folder = r"C:\Users\Hieu Pham\Downloads\test"
style = 'phim điện ảnh trung quốc, tên riêng và địa điểm 100% dùng hán việt'
target_language = 'Vietnamese' 

#Example
#historical drama
#modern drama
#news & current affairs

#The number of subtitle lines to be translated per API call. The code works stably with a value of 1.
#However, the drawback is that it results in a large number of API calls. I’m still working on improving the translation efficiency.
batch_size=10
context_window=50
concurrency = 30 #threading

In [2]:
#Split the file. The purpose is for translation. You can check how the file is split to understand better. 
#Note that after splitting, the original subtitle file will be deleted to avoid code conflicts when merging. 
#You can either save two versions in separate folders or modify the code so it doesn’t delete the original file.

import os
import re

def split_srt_folder(input_folder):
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.srt'):
            file_path = os.path.join(input_folder, file_name)
            base_name = os.path.splitext(file_name)[0]
            ts_file = os.path.join(input_folder, f"{base_name}_timestamps.txt")
            sub_file = os.path.join(input_folder, f"{base_name}_subtitles.txt")
            
            timestamps = []
            subtitles = []
            
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read().strip()
                blocks = re.split(r'\n\n', content)
                
                for block in blocks:
                    lines = block.split('\n')
                    if len(lines) >= 3:
                        index = lines[0].strip()
                        timestamp = lines[1].strip()
                        subtitle_text = ' '.join(lines[2:]).strip()
                        
                        timestamps.append(f"{index} {timestamp}")
                        subtitles.append(f"{index} {subtitle_text}")
            
            with open(ts_file, 'w', encoding='utf-8') as ts_out:
                ts_out.write('\n'.join(timestamps))
            
            with open(sub_file, 'w', encoding='utf-8') as sub_out:
                sub_out.write('\n'.join(subtitles))
            
            os.remove(file_path)
            print(f"Tách file hoàn tất: {file_name} -> {ts_file}, {sub_file} (Gốc đã xóa)")

split_srt_folder(input_folder)

Tách file hoàn tất: 1.srt -> C:\Users\Hieu Pham\Downloads\test\1_timestamps.txt, C:\Users\Hieu Pham\Downloads\test\1_subtitles.txt (Gốc đã xóa)


In [3]:
#main translation code cell
import os
import re
import time
import random
import json
import tempfile
from concurrent.futures import ThreadPoolExecutor, wait
from chat_bot import call_chatbot
from dotenv import load_dotenv

load_dotenv()
gpt_api_key = os.environ.get("CHATGPT_API_KEY")

def is_translatable(line: str) -> bool:
    s = line.strip()
    if not s:
        return False
    for ch in s:
        if ch.isalpha() or ch.isdigit():
            return True
    return False

def translate_file(
    file_path,
    style,
    batch_size=1,
    context_window=100,
    summary_lines_count=500,
    target_language='Vietnamese',
    input_encoding="utf-8",
    concurrency=10,
    max_retries=3,
    checkpoint_dir=None
):
    """
    Phiên bản an toàn: sau mỗi nhóm 'concurrency' batches sẽ persist:
      - ghi vào output_file, fsync để đảm bảo dữ liệu xuống đĩa
      - lưu checkpoint (atomic) chứa next_batch_index để resume
    checkpoint_dir: nếu None, lưu cùng thư mục với output_file
    """

    # clamp batch_size
    if batch_size < 1:
        batch_size = 1
    if batch_size > 20:
        batch_size = 20

    dir_name = os.path.dirname(file_path) or "."
    base_name = os.path.basename(file_path).rsplit('.', 1)[0]
    output_file = os.path.join(dir_name, f"{base_name}_translated.txt")

    if checkpoint_dir is None:
        checkpoint_dir = dir_name
    os.makedirs(checkpoint_dir, exist_ok=True)
    checkpoint_file = os.path.join(checkpoint_dir, f"{base_name}_translated.checkpoint.json")

    # load lines
    with open(file_path, "r", encoding=input_encoding) as f:
        lines = f.readlines()
    total = len(lines)

    # summary context (synchronous)
    initial_context_lines = lines[:summary_lines_count]
    initial_context_str = "".join(initial_context_lines)
    prompt_context = f"""
    Dựa trên {summary_lines_count} dòng đầu của file subtitle dưới đây, hãy tóm tắt bối cảnh của nội dung sau đây.

    Dữ liệu:
    {initial_context_str}
    """
    summary_context = call_chatbot(prompt_context, "gpt-4o-mini", "chatgpt", gpt_api_key)
    print("--- Summary context ---")
    print(summary_context)
    print("-----------------------")

    # prepare batches
    batches = []
    for start in range(0, total, batch_size):
        end = min(total, start + batch_size)
        batches.append((start, end))
    batch_count = len(batches)

    # resume: read checkpoint if exists
    next_batch_index = 0
    if os.path.exists(checkpoint_file):
        try:
            with open(checkpoint_file, "r", encoding="utf-8") as cf:
                ck = json.load(cf)
                # ck should contain next_batch (int)
                if isinstance(ck.get("next_batch"), int) and 0 <= ck["next_batch"] <= batch_count:
                    next_batch_index = ck["next_batch"]
                    print(f"[RESUME] Found checkpoint. Starting from batch index {next_batch_index} (batch {next_batch_index+1}/{batch_count}).")
                else:
                    print("[RESUME] Checkpoint file malformed -> ignoring and starting from 0.")
        except Exception as e:
            print(f"[RESUME] Failed to read checkpoint ({e}) -> starting from 0.")

    # helper: atomic write checkpoint
    def write_checkpoint(next_idx):
        tmp = checkpoint_file + ".tmp"
        payload = {"next_batch": next_idx}
        with open(tmp, "w", encoding="utf-8") as t:
            json.dump(payload, t)
            t.flush()
            os.fsync(t.fileno())
        os.replace(tmp, checkpoint_file)

    # helper: call for each batch (same as earlier)
    def make_prompt_and_call(start_idx, end_idx):
        batch_lines = lines[start_idx:end_idx]
        batch_indices = list(range(start_idx, end_idx))

        translatable_items = []
        for idx, ln in zip(batch_indices, batch_lines):
            if is_translatable(ln):
                translatable_items.append((idx, ln.rstrip("\n")))

        if not translatable_items:
            out_lines = []
            for idx in batch_indices:
                out_lines.append(f"{idx+1}\t\n")
            return (start_idx, "".join(out_lines), len(batch_indices))

        half_before = max(0, (context_window - len(translatable_items)) // 2)
        start_ctx = max(0, translatable_items[0][0] - half_before)
        end_ctx = min(total, start_ctx + context_window)
        if end_ctx - start_ctx < context_window:
            start_ctx = max(0, end_ctx - context_window)
        context_segment = "".join(lines[start_ctx:end_ctx])

        prompt_lines = []
        for idx, ln in translatable_items:
            prompt_lines.append(f"{ln}")
        prompt_batch = "\n\n".join(prompt_lines)

        prompt = f"""
        Summary (bối cảnh chung):
        {summary_context}

        Context tham khảo (khoảng {context_window} dòng, cố gắng đặt dòng/khối cần dịch ở giữa nếu có thể):
        {context_segment}

        Các dòng sau đây cần dịch sang {target_language} với phong cách "{style}":
        {prompt_batch}

        **Yêu cầu trả về:**
        - Mỗi dòng dịch trả về kèm index ở đầu, sau đó là bản dịch ngắn gọn.
        - Giữ đúng thứ tự dòng như input.
        - Dòng nào trống hoặc chỉ có dấu câu thì bỏ qua (không dịch, không trả).
        - Không thêm thắt, không giải thích, không kèm văn bản gốc.
        """

        attempt = 0
        while attempt < max_retries:
            try:
                translated_batch_text = call_chatbot(prompt, "gpt-5-mini", "chatgpt", gpt_api_key)
                return (start_idx, translated_batch_text.strip() + "\n", len(batch_indices))
            except Exception as e:
                attempt += 1
                wait_time = (2 ** attempt) + random.random()
                print(f"[WARN] Lỗi khi gọi API cho batch {start_idx+1}-{end_idx}. Attempt {attempt}/{max_retries}. Sleep {wait_time:.1f}s. Lỗi: {e}")
                time.sleep(wait_time)

        # nếu fail -> placeholder error lines
        err_text = ""
        for idx in batch_indices:
            err_text += f"{idx+1}\t[ERROR_TRANSLATION]\n"
        return (start_idx, err_text, len(batch_indices))

    processed = next_batch_index * batch_size  # approximate number of lines already done (for progress; approximate if last batch smaller)

    # Open output file in append mode if resuming, else write mode
    open_mode = "a" if next_batch_index > 0 and os.path.exists(output_file) else "w"
    with open(output_file, open_mode, encoding="utf-8") as f_out:
        with ThreadPoolExecutor(max_workers=concurrency) as executor:
            # iterate groups starting from next_batch_index
            group_start_idx = next_batch_index
            while group_start_idx < batch_count:
                group_end_idx = min(batch_count, group_start_idx + concurrency)
                group = batches[group_start_idx:group_end_idx]

                futures = {}
                for (start_idx, end_idx) in group:
                    fut = executor.submit(make_prompt_and_call, start_idx, end_idx)
                    futures[fut] = start_idx

                # wait all
                done, not_done = wait(futures.keys())
                # collect results
                results_map = {}
                for fut in done:
                    try:
                        s_idx, out_text, cnt = fut.result()
                        results_map[s_idx] = (out_text, cnt)
                    except Exception as e:
                        st = futures[fut]
                        batch_len = batches[[b[0] for b in batches].index(st)][1] - st
                        out_text = ""
                        for idx in range(st, st + batch_len):
                            out_text += f"{idx+1}\t[ERROR]\n"
                        results_map[st] = (out_text, batch_len)
                        print(f"[ERROR] Future failed for batch starting at {st+1}: {e}")

                # write group results in batch order and persist
                for (start_idx, end_idx) in group:
                    out_text, cnt = results_map.get(start_idx, ("", end_idx - start_idx))
                    f_out.write(out_text)
                    processed += cnt

                # flush + fsync to ensure durable write
                try:
                    f_out.flush()
                    os.fsync(f_out.fileno())
                except Exception as e:
                    print(f"[WARN] fsync failed: {e}")

                # update checkpoint: next batch to process
                next_group_batch_index = group_end_idx
                try:
                    write_checkpoint(next_group_batch_index)
                except Exception as e:
                    print(f"[WARN] Việc ghi checkpoint thất bại: {e}")

                print(f"Đã xử lý ~{processed}/{total} dòng... (đã hoàn thành nhóm batches {group_start_idx+1}..{group_end_idx})")
                group_start_idx = group_end_idx

    print(f"Dịch xong! File đã lưu tại: {output_file}")
    # optional: remove checkpoint on full success
    try:
        if os.path.exists(checkpoint_file):
            with open(checkpoint_file, "r", encoding="utf-8") as cf:
                ck = json.load(cf)
                if ck.get("next_batch") == batch_count:
                    os.remove(checkpoint_file)
    except Exception:
        pass



def translate_folder(input_folder, style, batch_size=1, context_window=100, summary_lines_count=500, target_language='Vietnamese', concurrency=10):
    if not os.path.exists(input_folder):
        print(f"Thư mục {input_folder} không tồn tại.")
        return

    files = [f for f in os.listdir(input_folder) if f.endswith("subtitles.txt")]
    if not files:
        print("Không tìm thấy file subtitles nào trong thư mục.")
        return

    for file in files:
        file_path = os.path.join(input_folder, file)
        print(f"Bắt đầu dịch file: {file}")
        translate_file(file_path, style, batch_size=batch_size, context_window=context_window, summary_lines_count=summary_lines_count, target_language=target_language, concurrency=concurrency)
        print(f"Hoàn thành dịch file: {file}\n")
        
# Giả sử input_folder đã được định nghĩa trước đó
translate_folder(
    input_folder=input_folder, 
    style=style, 
    batch_size=batch_size, 
    context_window=context_window, 
    target_language=target_language,
    concurrency=concurrency
)

c:\Users\Hieu Pham\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Bắt đầu dịch file: 1_subtitles.txt
--- Summary context ---
Dựa vào 500 dòng đầu của tệp phụ đề, bối cảnh nội dung có thể được tóm tắt như sau:

Câu chuyện xoay quanh Li NantIng, một chàng trai mù và là con trai của một gia đình quý tộc giàu có, đang tới để đón cô dâu mới của mình, Shen Wei Yu. Cuộc hôn nhân này được basing từ một thỏa thuận hôn nhân trước đó giữa gia đình Li và gia đình Shen. Tuy nhiên, bối cảnh trở nên phức tạp khi gia đình Shen có hai cô con gái: Shen Wei Yu (cô dâu) bị khuyết tật và hiếm khi ra ngoài, và em gái Shen Ling vẫn còn đang học đại học.

Có sự phản đối và tranh cãi giữa các thành viên trong gia đình Li và Shen về cuộc hôn nhân này, với nhiều nhận xét khinh miệt từ những người xung quanh, đặc biệt về việc Li NantIng là một chàng trai mù. Trong khi đó, Shen Wei Yu cũng có những toan tính riêng, muốn lợi dụng sự kết nối với gia đình Li để củng cố địa vị của mình, mặc dù cô thực sự có cảm xúc phức tạp và không hoàn toàn hài lòng với tình huống hiện tại.

Khung

In [4]:
#clean
import re

def clean_subtitle_file(file_path):
    cleaned_lines = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:  # bỏ dòng trống
                continue

            # Chuẩn hóa: "5.abc", "5. abc", "5 abc" => "5 abc"
            line = re.sub(r"^(\d+)\s*\.?\s*", r"\1 ", line)
            cleaned_lines.append(line)

    # Ghi đè file cũ
    with open(file_path, "w", encoding="utf-8") as f:
        f.write("\n".join(cleaned_lines))


file = r"C:\Users\Hieu Pham\Downloads\test\1_subtitles_translated.txt"
clean_subtitle_file(file)

In [5]:
#code to clean and merge the timestamp file and the translated file into a complete SRT subtitle file
                
def merge_srt_folder(input_folder):
    for file_name in os.listdir(input_folder):
        if file_name.endswith('_timestamps.txt'):
            base_name = file_name.replace('_timestamps.txt', '')
            ts_file = os.path.join(input_folder, file_name)
            sub_file = os.path.join(input_folder, f"{base_name}_subtitles_translated.txt")
            output_file = os.path.join(input_folder, f"{base_name}_merged.srt")

            if os.path.exists(sub_file):
                timestamps = {}
                subtitles = {}

                # Đọc file timestamps
                with open(ts_file, 'r', encoding='utf-8') as ts_in:
                    for line in ts_in:
                        parts = line.strip().split(maxsplit=1)  # Tách tối đa 1 lần
                        if len(parts) == 2:
                            timestamps[parts[0]] = parts[1]

                # Đọc file subtitles
                with open(sub_file, 'r', encoding='utf-8') as sub_in:
                    for line in sub_in:
                        parts = line.strip().split(maxsplit=1)  # Tách tối đa 1 lần
                        if len(parts) == 2:
                            subtitles[parts[0]] = parts[1]

                # Ghi file .srt
                with open(output_file, 'w', encoding='utf-8') as out:
                    for index in sorted(timestamps.keys(), key=int):
                        timestamp = timestamps[index]
                        subtitle = subtitles.get(index, " ")  # Nếu không có sub, để khoảng trắng
                        out.write(f"{index}\n{timestamp}\n{subtitle}\n\n")

                print(f"Gộp file hoàn tất: {output_file}")



merge_srt_folder(input_folder)

Gộp file hoàn tất: C:\Users\Hieu Pham\Downloads\test\1_merged.srt


The following code snippets are intended for fine-tuning when converting from SRT to an audio file. You can check the corresponding project in another repo of mine.
Use with caution — testing is required first.
Merging timestamps means combining adjacent timestamps that are too close together, in order to prevent the voice from being cut off due to subtitles being too short.

In [6]:
import os
import re
from datetime import timedelta

def parse_time(time_str):
    """Chuyển đổi chuỗi thời gian SRT sang đối tượng timedelta."""
    parts = re.match(r'(\d{2}):(\d{2}):(\d{2}),(\d{3})', time_str.strip())
    if not parts:
        raise ValueError(f"Định dạng thời gian không hợp lệ: {time_str}")
    hours, minutes, seconds, milliseconds = map(int, parts.groups())
    return timedelta(hours=hours, minutes=minutes, seconds=seconds, milliseconds=milliseconds)

def format_time(td):
    """Chuyển đổi đối tượng timedelta sang chuỗi thời gian SRT."""
    if td.total_seconds() < 0:
         return f"00:00:00,000" # Tránh thời gian âm trong SRT

    total_seconds = td.total_seconds()
    hours = int(total_seconds // 3600)
    minutes = int((total_seconds % 3600) // 60)
    seconds = int(total_seconds % 60)
    milliseconds = int(td.microseconds // 1000)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d},{milliseconds:03d}"

def process_srt_file(file_path, gap_ms=100):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file {file_path}")
        return
    except Exception as e:
        print(f"Lỗi khi đọc file {file_path}: {e}")
        return

    subs = []
    raw_blocks = re.split(r'\r?\n\s*\r?\n', content.strip())
    seq_counter = 1

    for block_num, block_content in enumerate(raw_blocks):
        if not block_content.strip():
            continue

        block_pattern = re.match(
            r'(\d+)\s*\r?\n'
            r'(\d{2}:\d{2}:\d{2},\d{3})\s*-->\s*(\d{2}:\d{2}:\d{2},\d{3})\s*\r?\n?'
            r'(.*)',
            block_content,
            re.DOTALL
        )

        if block_pattern:
            original_seq = block_pattern.group(1)
            start_time_str = block_pattern.group(2)
            end_time_str = block_pattern.group(3)
            text = block_pattern.group(4).strip() # .strip() quan trọng ở đây

            try:
                start_time = parse_time(start_time_str)
                end_time = parse_time(end_time_str)
                if end_time < start_time:
                    # print(f"Cảnh báo file {file_path}, sub gốc {original_seq}: end_time < start_time. Sửa end_time = start_time.")
                    end_time = start_time
                
                subs.append({
                    'seq': str(seq_counter),
                    'start_time': start_time,
                    'end_time': end_time,
                    'text': text, # text đã được .strip()
                    'original_start_str': start_time_str,
                })
                seq_counter += 1
            except ValueError as e:
                print(f"Lỗi khi xử lý thời gian trong file {file_path}, sub gốc {original_seq} (khối {block_num+1}): {e}")
                continue
        else:
            print(f"Cảnh báo: Không thể parse khối sub trong {file_path} (khối {block_num+1}): \"{block_content[:100]}...\"")

    if not subs:
        print(f"Không tìm thấy sub nào hợp lệ trong file {file_path} hoặc định dạng không đúng.")
        return

    # Điều chỉnh end_time
    for i in range(len(subs) - 1):
        current_sub = subs[i]
        next_sub = subs[i+1]
        target_end_time = next_sub['start_time'] - timedelta(milliseconds=gap_ms)
        if target_end_time > current_sub['start_time']:
            current_sub['end_time'] = target_end_time
        else:
            current_sub['end_time'] = current_sub['start_time'] + timedelta(milliseconds=1)
    
    if subs:
        last_sub = subs[-1]
        if last_sub['end_time'] < last_sub['start_time']:
            last_sub['end_time'] = last_sub['start_time'] + timedelta(milliseconds=1)

    # Ghi đè file cũ với logic dòng trống đã sửa
    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            srt_output_lines = []
            num_subs = len(subs)
            for i, sub_item in enumerate(subs):
                srt_output_lines.append(f"{sub_item['seq']}")
                srt_output_lines.append(f"{sub_item['original_start_str']} --> {format_time(sub_item['end_time'])}")
                
                # Thêm dòng text (có thể là chuỗi rỗng "", sẽ được join thành một dòng trống)
                # sub_item['text'] đã được strip() trong quá trình parsing.
                # Nếu text có nhiều dòng, các \n bên trong sẽ được giữ nguyên.
                srt_output_lines.append(sub_item['text'])
                
                # Thêm dòng trống separator nếu không phải sub cuối cùng
                if i < num_subs - 1:
                    # Luôn thêm một dòng trống làm separator chính.
                    # Nếu sub_item['text'] là rỗng (""), dòng srt_output_lines.append(sub_item['text']) ở trên
                    # đã tạo ra dòng trống thứ nhất (cho nội dung). Dòng trống này là dòng thứ hai (separator).
                    # Nếu sub_item['text'] có nội dung, đây sẽ là dòng trống duy nhất sau text.
                    srt_output_lines.append("") 
            
            final_srt_string = "\n".join(srt_output_lines)
            
            # Đảm bảo file kết thúc bằng một ký tự newline duy nhất (nếu file không rỗng)
            if final_srt_string:
                final_srt_string = final_srt_string.rstrip('\r\n') + '\n'
            
            f.write(final_srt_string)
        print(f"Đã xử lý và ghi đè file: {file_path}")
    except Exception as e:
        print(f"Lỗi khi ghi file {file_path}: {e}")

def process_srt_folder(folder_path, gap_ms=100):
    if not os.path.isdir(folder_path):
        print(f"Lỗi: Thư mục '{folder_path}' không tồn tại.")
        return
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(".srt"):
            file_path = os.path.join(folder_path, filename)
            print(f"Đang xử lý file: {file_path}")
            process_srt_file(file_path, gap_ms)
    print("Hoàn tất xử lý thư mục.")


desired_gap_ms = 100
process_srt_folder(input_folder, desired_gap_ms)

Đang xử lý file: C:\Users\Hieu Pham\Downloads\test\1_merged.srt
Đã xử lý và ghi đè file: C:\Users\Hieu Pham\Downloads\test\1_merged.srt
Hoàn tất xử lý thư mục.
